# Шумоподавляющий автокодировщик

Denoising autoencoder (DAE)

Задание: создать модель для подавления шумов в медицинских изображениях.

В этой работе мы повторим идею из статьи "Medical image denoising using convolutional denoising autoencoders".

In [ ]:
# код для варианта

name = "" # Впишите ваше ФИО

def calculate_variant(name):
    return sum(ord(char) for char in name) % 3 + 1

print(f"Ваш вариант - №{calculate_variant(name)}")

## Варианты

1. OrganSMNIST
1. ChestMNIST
1. PneumoniaMNIST

## Порядок выполнения

1. Загрузить датасет по варианту;
1. Провести предварительную обработку данных;
1. Используя фреймворк PyTorch создать модель;
1. Обучить модель;
1. Попробовать подобрать гиперпараметры;
1. Оценить результаты лучшей модели на тестовой выборке.

## Источники

1. [Medical image denoising using convolutional denoising autoencoders](https://arxiv.org/pdf/1608.04667.pdf)
1. [Автоэнкодеры: типы архитектур и применение](https://neurohive.io/ru/osnovy-data-science/avtojenkoder-tipy-arhitektur-i-primenenie/)
1. [Автокодировщик](https://neerc.ifmo.ru/wiki/index.php?title=%D0%90%D0%B2%D1%82%D0%BE%D0%BA%D0%BE%D0%B4%D0%B8%D1%80%D0%BE%D0%B2%D1%89%D0%B8%D0%BA)

## Статьи для примера

1. [LLNet: A Deep Autoencoder approach to Natural Low-light Image Enhancement](https://arxiv.org/pdf/1511.03995.pdf)
1. [Deep Learning on Image Denoising: An Overview](https://arxiv.org/pdf/1912.13171.pdf)
1. [Boltzmann Machines and Denoising Autoencoders for Image Denoising](https://arxiv.org/pdf/1301.3468.pdf)
1. [Denoising Vanilla Autoencoder for RGB and GS Images with Gaussian Noise](https://app.dimensions.ai/details/publication/pub.1165117438)

## Импортирование модулей

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import make_grid
import matplotlib.pyplot as plt

%matplotlib inline

В этой работе можете попробовать задействовать GPU для ускорения вычислений. [How To Use GPU with PyTorch](https://wandb.ai/wandb/common-ml-errors/reports/How-To-Use-GPU-with-PyTorch---VmlldzozMzAxMDk)

In [ ]:
if torch.cuda.is_available():
  device = torch.device('cuda')
  print('Работаем на GPU')
else:
  device = torch.device('cpu')
  print('Работаем на CPU')

Изначально тензоры создаются на CPU, и с помощью метода `.to(device)` на тензорах и моделях вы можете переносить их с устройства на устройство. Если в результате вычислений создаются новые тензоры, то они уже создаются на устройствах, на которых производились эти вычисления.

## Загрузка данных

Ссылка на подборку датасетов - https://medmnist.com/. По этой ссылке вы найдете описание самих датасетов, а также инструкцию по установке библиотеки и созданию объекта DataClass и далее DataLoader.

[Официальный пример](https://github.com/MedMNIST/MedMNIST/blob/main/examples/getting_started.ipynb) как пользоваться датасетом и как загрузить его данные от самих разработчиков.

In [ ]:
#
# Ваш код для DataSet и DataLoader
#

**Какие преобразования вы применили при создании DataSet и почему?**

Ваш ответ:

## Создание модели

Автокодировщик (автоэнкодер) состоит из двух частей и вывод первой части является входом второй части. Строение модели вы можете посмотреть в статье в описании и воспроизвести ее. Создание слоев свертки вам знакомо с предыдущей работы, а для декодера вам потребуется двумерная транспонированная свертка (ConvTranspose2d).

In [ ]:
# кодировщик
class Encoder(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv = nn.Sequential(
        #
        # ваш код
        #
    )

  def forward(self, x):
    return output  # здесь на выходе может быть произвольный тензор


#  декодер
class Decoder(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv = nn.Sequential(
        #
        # ваш код
        #
    )

  def forward(self, x):
    # Помните, что здесь на входе должен приниматься тензор размерностью как у кодировщика на выходе
    #
    # ваш код
    #
    return output  # На выходе должен быть тензор размерностью как исходное изображение.

**В чем разница между сверткой и транспонированной сверткой?**

Ваш ответ:

Далее кодировщик и декодировщик мы объединяем в автокодировщик.

In [ ]:
class Autoencoder(nn.Module):
  def __init__(self, encoder, decoder):
    super().__init__()
    self.encoder = encoder
    # self.encoder.to(device)

    self.decoder = decoder
    # self.decoder.to(device)

  def forward(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

## Обучение

В качестве оптимизатора можете использовать SGD или Adam или в целом попробовать другие оптимизаторы, что предлагает pytorch.

В качестве функции потерь воспользуйтесь MSE (MSELoss) или другой функцией, которая подходит для сравнения изображений.

In [ ]:
log_dict = {
    'training_loss_per_batch': [],
    'validation_loss_per_batch': [],
}

for epoch in range(epochs):

  print(f'Эпоха {epoch+1}/{epochs}')
  train_losses = []

  print('обучение...')
  model.train()
  for images in train_loader:

    # Шаги для обучения:
    # 1. загрузить изображения
    # 2. добавить гауссовский шум шум (torch.normal() и clip() / clamp())
    # 3. реконструировать изображение (прямой проход через энкодер и декодер)
    # 4. рассчитать `loss`
    # 5. выполнить `loss.backward ()` для вычисления градиентов функции потери относительно параметров модели
    # 6. выполнить шаг оптимизации `optimizer.step ()`
    # 7. занулить градиенты

    #
    # ваш код
    #

    log_dict['training_loss_per_batch'].append(loss.item())

  # Валидация
  print('валидация...')
  model.eval()
  for val_images in val_loader:
    with torch.no_grad():
      #
      # ваш код
      #

      log_dict['validation_loss_per_batch'].append(val_loss.item())

**Что делает clip() или clamp() и почему его требуется использовать после добавления шума?**

Ваш ответ:

## Контроль результатов и экспериментов

Вы всегда должны следить за тем как идет обучение и как меняются метрики в его процессе. В этой работе вы можете воспользоваться [TensorBoard](https://pytorch.org/docs/stable/tensorboard.html) или создать переменную со списком и в него складывать значения с каждых n итераций и с помощью matplotlib строить графики.

Далее приведены части кода, которые позволят в Google Colab воспользоваться TensorBoard для PyTorch.

In [ ]:
%load_ext tensorboard  # Загружает один раз расширение для ноутбука. Эту команду можно перенести в самое начало.
%tensorboard --logdir runs # Запускает сам tensorboard в выводе текущей ячейки. Лучше перенести в самый конец.

from torch.utils.tensorboard import SummaryWriter # Импорт модуля

writer = SummaryWriter()  # создание экземпляра класса, который отвечает за ведение логов.

for n_iter in range(100): # в цикле обучения вы используете метод add_scalar или другие для добавления записи в логи.
    writer.add_scalar('Loss/train', np.random.random(), n_iter)  # пример вызова самого метода.

**По графикам потерь и метрике сделайте вывод была ли модель обучена, недообучена или переобучена.**

Ваш ответ:

**Если модель недообучена или переобучена, то что могло послужить причинами этого?**

Ваш ответ:

## Результат работы модели

Выведите итоговую метрику для тестовых изображений. И отобразите несколько примеров работы модели: исходное изображение и с убранным шумом.

Пример для цветных изображений.

```python
test_images = test_images.to(device)
with torch.no_grad():
  reconstructed_imgs = network(test_images)
reconstructed_imgs = reconstructed_imgs.cpu()
test_images = test_images.cpu()
imgs = torch.stack([test_images.view(-1, 3, 32, 32), reconstructed_imgs], dim=1).flatten(0,1)
grid = make_grid(imgs, nrow=10, normalize=True, padding=1)
grid = grid.permute(1, 2, 0)
plt.figure(dpi=170)
plt.title('Original/Reconstructed')
plt.imshow(grid)
log_dict['visualizations'].append(grid)
plt.axis('off')
plt.show()
```

In [ ]:
#
# Ваш код
#

## Контрольные вопросы

**В чем особенность архитектуры автокодировщиков?**

Ваш ответ:

**Какие метрики можно использовать для оценки качества изображений?**

Ваш ответ:
